In [ ]:
# ✅ Final Hybrid CNN + ViT Pneumonia Detection Notebook



## ✅ 1. Imports

import os
import zipfile
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, ConcatDataset, WeightedRandomSampler
from torchvision import datasets, transforms, models

from sklearn.metrics import classification_report, confusion_matrix
from transformers import ViTModel


In [ ]:
## ✅ 2. CLAHE Preprocessing Class

class CLAHETransform:
    def __call__(self, img):
        img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        cl_img = clahe.apply(img_cv)
        return Image.fromarray(cl_img).convert('RGB')


In [ ]:
## ✅ 3. Define Transforms (Training & Evaluation)
train_transform = transforms.Compose([
    CLAHETransform(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    CLAHETransform(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## ✅ 4. Extract and Load Dataset


import os
import zipfile

zip_path = "/content/drive/MyDrive/manas/Copy of Final_Chest_Xray_Dataset_ZIP.zip"
extract_path = "/content/drive/MyDrive/manas"

# --- Recommended Code ---
# First, ensure the extraction path directory exists
os.makedirs(extract_path, exist_ok=True)

# Now, run the extraction without the 'if' condition
print(f"Attempting to extract '{zip_path}' to '{extract_path}'...")
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("✅ Extraction completed successfully!")
except FileNotFoundError:
    print(f"❌ ERROR: The file was not found at '{zip_path}'. Please check the path and file name.")
except Exception as e:
    print(f"❌ An error occurred: {e}")


Attempting to extract '/content/drive/MyDrive/manas/Copy of Final_Chest_Xray_Dataset_ZIP.zip' to '/content/drive/MyDrive/manas'...
✅ Extraction completed successfully!


In [ ]:
## ✅ 5. Prepare Datasets and Loaders

custom_data =  "/content/drive/MyDrive/manas"

# ✅ Load datasets
train_dir = os.path.join(custom_data, 'train')
val_dir = os.path.join(custom_data, 'val')
test_dir = os.path.join(custom_data, 'test')


train_ds = datasets.ImageFolder(train_dir, transform=train_transform)
val_ds = datasets.ImageFolder(val_dir, transform=test_transform)
test_ds = datasets.ImageFolder(test_dir, transform=test_transform)

  # ✅ Dataloaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)



In [ ]:
## ✅ 6. Define Hybrid CNN + ViT Model

class HybridCNNViT(nn.Module):
    def __init__(self):
        super(HybridCNNViT, self).__init__()
        densenet = models.densenet121(pretrained=True)
        self.cnn_features = densenet.features
        self.cnn_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.vit = ViTModel.from_pretrained('google/vit-base-patch16-224')
        for param in self.vit.parameters():
            param.requires_grad = False

        self.fc = nn.Sequential(
            nn.Linear(1024 + 768, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 2)
        )

    def forward(self, x):
        cnn_out = self.cnn_features(x)
        cnn_out = self.cnn_pool(cnn_out).view(x.size(0), -1)
        vit_out = self.vit(pixel_values=x).last_hidden_state[:, 0, :]
        out = torch.cat((cnn_out, vit_out), dim=1)
        return self.fc(out)




In [ ]:

## ✅ 7. Training Setup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hybrid_model = HybridCNNViT().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(hybrid_model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 131MB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Huggin

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
## ✅ 8. Training Loop

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=10):
    best_val_loss = float('inf')
    patience, counter = 5, 0

    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total

        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_correct += (outputs.argmax(1) == labels).sum().item()
                val_total += labels.size(0)

        val_acc = 100 * val_correct / val_total
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}: Train Acc = {train_acc:.2f}%, Val Acc = {val_acc:.2f}%, Val Loss = {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping.")
                break

train_model(hybrid_model, train_loader, val_loader, criterion, optimizer, scheduler, device)




Epoch 1: Train Acc = 95.00%, Val Acc = 97.60%, Val Loss = 3.5323
Epoch 2: Train Acc = 97.33%, Val Acc = 97.95%, Val Loss = 3.3493
Epoch 3: Train Acc = 97.84%, Val Acc = 99.06%, Val Loss = 2.0543
Epoch 4: Train Acc = 98.70%, Val Acc = 98.48%, Val Loss = 2.2536
Epoch 5: Train Acc = 98.60%, Val Acc = 98.13%, Val Loss = 2.9303
Epoch 6: Train Acc = 98.95%, Val Acc = 98.54%, Val Loss = 2.5426
Epoch 7: Train Acc = 99.46%, Val Acc = 99.12%, Val Loss = 1.7771
Epoch 8: Train Acc = 99.79%, Val Acc = 99.53%, Val Loss = 0.8328
Epoch 9: Train Acc = 99.72%, Val Acc = 99.53%, Val Loss = 1.4299
Epoch 10: Train Acc = 99.79%, Val Acc = 99.42%, Val Loss = 1.7458


In [ ]:
 #After training ends
torch.save(hybrid_model.state_dict(), "model_checkpoint_epoch10.pth")

In [ ]:
from google.colab import files
files.download("model_checkpoint_epoch10.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
## ✅ 9. Final Evaluation on Test Set

hybrid_model.load_state_dict(torch.load('best_model.pth'))
hybrid_model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = hybrid_model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("\n✅ Classification Report:")
print(classification_report(all_labels, all_preds, digits=2))

print("\n✅ Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))





✅ Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       855
           1       1.00      1.00      1.00       855

    accuracy                           1.00      1710
   macro avg       1.00      1.00      1.00      1710
weighted avg       1.00      1.00      1.00      1710


✅ Confusion Matrix:
[[854   1]
 [  2 853]]
